In [ ]:
#Convert npy to csv, ready for pandas manipulation

import numpy as np
import pandas as pd

train = np.load('ecs171train.npy')
test = np.load('ecs171test.npy')
np.savetxt('pre_test.csv', test, fmt='%s')
np.savetxt('pre_train.csv', train, fmt='%s')



delete_list = ["b'", "'","\n"]
tmp = open("pre_test.csv")
tmp1 =open("pre_train.csv")
out = open("test.csv", "w+")
out1 = open("train.csv", "w+")

for line in tmp:
    for word in delete_list:
        line = line.replace(word, "")
    out.write(line)

tmp.close()   
out.close()

for line in tmp1:
    for word in delete_list:
        line = line.replace(word, "")
    out1.write(line)

tmp1.close()
out1.close()

li = 'id'
for i in range(1,770):
    li += ','+'f'+str(i) + ''


def line_prepender(filename, line):
    with open(filename, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write(line.rstrip('\r\n') + '\n' + content)

line_prepender("test.csv",li)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler,RobustScaler
from scipy import stats
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
import re
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score,log_loss, mean_squared_error,mean_absolute_error,confusion_matrix
from xgboost import XGBRegressor

%matplotlib inline

real_test = pd.read_csv("test.csv",dtype= np.float32)
real_train = pd.read_csv("train.csv")

In [ ]:
train = real_train.copy()
test = real_test.copy()  
#extract member id
test_member_id = test[test.columns[0]]
test = test.drop('id',axis=1)
test = test.astype('float32')

In [ ]:
x_regression_train = train.loc[train['loss'] > 0]


y_train = train.loss
#binarize data
y_train = (y_train > 0.0).astype(int)

y_regression_train = x_regression_train.loss

x_train = train.drop(['id','loss'], axis=1)
x_regression_train = x_regression_train.drop(['id','loss'], axis=1)

x_train.columns = test.columns
x_regression_train.columns = test.columns


#Preprocessing: 1. Drop cols of NA 2. Replace inf with NA 3.Replace NA with median
test= test.dropna(axis=1,how='all')
x_train= x_train.dropna(axis=1,how='all')
x_regression_train = x_regression_train.dropna(axis=1,how='all')


x_train.replace([np.inf, -np.inf], np.nan,inplace= True)
x_regression_train.replace([np.inf, -np.inf], np.nan,inplace= True)
test.replace([np.inf, -np.inf], np.nan,inplace= True)

x_train.fillna(x_train.median(),inplace=True)
x_regression_train.fillna(x_regression_train.median(),inplace=True)
test.fillna(test.median(),inplace=True)

x_regression_train.head() 
# both x_train ad test have 769 cols

In [ ]:
X_train_corr = x_train.copy()
test_copy = test.copy()
x_regression = x_regression_train.copy()
# Remove duplicate cols by deleting one of every pair of columns that has correlation 
# of 1 in the upper correlation matrix. From 758 cols down to 675 columns.
corr_matrix = X_train_corr.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] == 1.0)] 

X_train_corr = X_train_corr.drop(X_train_corr[to_drop], axis=1)
test_copy = test_copy.drop(test_copy[to_drop], axis=1)
x_regression = x_regression.drop(x_regression[to_drop],axis=1)

In [ ]:
x_regression = x_regression.drop(x_regression.std()[(x_regression.std() == 0)].index, axis=1)
X_train_corr = X_train_corr.drop(X_train_corr.std()[(X_train_corr.std() == 0)].index, axis=1)
test_copy = test_copy.drop(test_copy.std()[(test_copy.std()==0)].index,axis = 1)

In [ ]:
corr_matrix = X_train_corr.corr().abs()
indices = np.where(corr_matrix > 0.996)
indices = [(corr_matrix.index[x], corr_matrix.columns[y]) for x, y in zip(*indices)
                                        if x != y and x < y]

i = 0
for pairs in indices:
    X_train_corr[pairs[1]+"-"+pairs[0]] = X_train_corr[pairs[1]] - X_train_corr[pairs[0]]
    test_copy[pairs[1]+"-"+pairs[0]] = test_copy[pairs[1]] - test_copy[pairs[0]]
    x_regression[pairs[1]+"-"+pairs[0]] = x_regression[pairs[1]] - x_regression[pairs[0]]
    i+=1
    
    
corr_matrix1 = X_train_corr.corr().abs()
upper = corr_matrix1.where(np.triu(np.ones(corr_matrix1.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.99)]

X_train_corr = X_train_corr.drop(X_train_corr[to_drop], axis=1)
test_copy = test_copy.drop(test_copy[to_drop], axis=1)
x_regression = x_regression.drop(x_regression[to_drop],axis=1)

#355 cols
print('GD features',i)

In [ ]:
Regressor_model = XGBRegressor()
Regressor_model.fit(x_regression, y_regression_train , verbose = False)
predicted_output = Regressor_model.predict(test_copy)

XGBoost_Submission = pd.DataFrame({'id': test_member_id , 'loss': predicted_output})
XGBoost_Submission.to_csv('regression.csv' , index = False)

In [ ]:
X = RobustScaler().fit_transform(x_train)
Test = RobustScaler().fit_transform(test)

Y = y_train.values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y ,test_size=0.30,
                                                   random_state=3)

In [ ]:
clf = xgb.sklearn.XGBClassifier(
    objective="binary:logistic", 
    learning_rate=0.05, 
    seed=9616, 
    max_depth=20, 
    gamma=10, 
    n_estimators=500)

eval_set = [(X_train, Y_train), (X_test, Y_test)]

clf.fit(X_train, Y_train, early_stopping_rounds=10, eval_metric=["error", "logloss"], eval_set=eval_set)

In [ ]:
Final_prediction = clf.predict(Test)

submission = pd.DataFrame({
        "id": test_member_id,
        "loss": Final_prediction
    })
submission.to_csv('classification.csv', index=False)

In [ ]:
classification_preds = pd.read_csv('classification.csv')

regression_preds = pd.read_csv('regression.csv')

#Replace values >0 in classification output with values in regression output
classification_preds  = regression_preds.mask(sub['loss']==0,classification_preds )
sub.to_csv("submission.csv",index=False)